In [1]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [2]:
import pandas as pd
df = pd.read_csv('train.csv')
#dataset link: https://www.kaggle.com/datasets/sdolezel/black-friday?resource=download

In [3]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [4]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI
#use langchain-experimental==0.0.49 version

In [5]:
agent = create_csv_agent(
    OpenAI(temperature=0),
    "train.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [13]:
agent

AgentExecutor(verbose=True, agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'df_head': '|    |   User_ID | Product_ID   | Gender   | Age   |   Occupation | City_Category   | Stay_In_Current_City_Years   |   Marital_Status |   Product_Category_1 |   Product_Category_2 |   Product_Category_3 |   Purchase |\n|---:|----------:|:-------------|:---------|:------|-------------:|:----------------|:-----------------------------|-----------------:|---------------------:|---------------------:|---------------------:|-----------:|\n|  0 |   1000001 | P00069042    | F        | 0-17  |           10 | A               | 2                            |                0 |                    3 |                  nan |                  nan |       8370 |\n|  1 |   1000001 | P00248942    | F        | 0-17  |           10 | A               | 2                            |                0 |                    1 |                  

In [14]:
agent.agent.llm_chain.prompt.template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df_head}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

In [6]:
agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)
Observation: 550068
Thought: I now know the final answer
Final Answer: There are 550068 rows in the dataframe.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 550068 rows in the dataframe.'}

In [ ]:
agent.run('How many people are female?')



> Entering new AgentExecutor chain...


C:\Users\rajan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Thought: I need to count the number of rows where the Gender column is equal to "F".
Action: python_repl_ast
Action Input: df[df['Gender'] == 'F'].count()
Observation: User_ID                       135809
Product_ID                    135809
Gender                        135809
Age                           135809
Occupation                    135809
City_Category                 135809
Stay_In_Current_City_Years    135809
Marital_Status                135809
Product_Category_1            135809
Product_Category_2             91530
Product_Category_3             37594
Purchase                      135809
dtype: int64
Thought: I now know the final answer
Final Answer: 135809

> Finished chain.


'135809'

In [ ]:
agent.run("how many people have stayed more than 3 years in the city and are male?")



> Entering new AgentExecutor chain...


In [ ]:
agent.run("Give me the count of number of males and females? Then, tell me which is more")



> Entering new AgentExecutor chain...
Thought: I need to use the value_counts() function to get the count of each gender, and then compare the values to determine which is more.
Action: python_repl_ast
Action Input: df['Gender'].value_counts()
Observation: Gender
M    414259
F    135809
Name: count, dtype: int64
Thought: Now I need to compare the values to determine which is more.
Action: python_repl_ast
Action Input: df['Gender'].value_counts().max()
Observation: 414259
Thought: I now know the final answer
Final Answer: There are 414259 males and 135809 females in the dataframe. There are more males than females.

> Finished chain.


'There are 414259 males and 135809 females in the dataframe. There are more males than females.'

In [ ]:
agent.run("what are the column names?")



> Entering new AgentExecutor chain...
Thought: I need to access the dataframe's column names
Action: python_repl_ast
Action Input: df.columns
Observation: Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')
Thought: I now know the final answer
Final Answer: The column names are 'User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3', and 'Purchase'.

> Finished chain.


"The column names are 'User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3', and 'Purchase'."

In [ ]:
agent.invoke('Which country appears the most and how many times it appears?')



> Entering new AgentExecutor chain...
Thought: I need to group the dataframe by the "City_Category" column and then count the number of occurrences for each group.
Action: python_repl_ast
Action Input: df.groupby("City_Category").size()
Observation: City_Category
A    147720
B    231173
C    171175
dtype: int64
Thought: I now know the final answer
Final Answer: The country that appears the most is B with 231173 occurrences.

> Finished chain.


{'input': 'Which country appears the most and how many times it appears?',
 'output': 'The country that appears the most is B with 231173 occurrences.'}

Microsoft Excel loader 

In [7]:
data = pd.read_excel("CollegeList.xlsx", engine='openpyxl')

In [8]:
data.to_csv("CollegeList.csv", index=False)

In [15]:
data.iloc[1: , :]

,University Name,Course,Link to that course,Price
1,Monash University,Master of Data Science,Master of Data Science | Study at Monash Unive...,49500.0
2,University of Melbourn,Master of Data Science,Master of Data Science - The University of Mel...,52256.0
3,Australian National University,Master of Mahine Learning and Computer Vision,Master of Machine Learning and Computer Vision...,40000.0
4,Australian National University,Master of Applied Data Analytics (1.5 years),Master of Applied Data Analytics - ANU,40000.0
5,University of New South Wales,Master of Data Science and Decision,Master of Data Science and Decisions (unsw.edu...,35000.0
6,University of Sydney,Master of Data Science,Master of Data Science - The University of Sydney,53000.0
7,University of Queensland,Master of Data Science,Master of Data Science - my.UQ - The Universit...,39000.0
8,University of Adelaide,Master of Data Science,Master of Data Science | Degree Finder (adelai...,51000.0
9,RMIT University,Master of Data Science,https://www.rmit.edu.au/study-with-us/levels-o...,39000.0


In [11]:
agent = create_csv_agent(OpenAI(temperature=0), 
                         'CollegeList.csv', 
                         verbose=True)

In [13]:
agent.invoke("Give me the names of Universities where Master of Data Science course is under the price of 40000")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the course is "Master of Data Science" and the price is less than 40000.
Action: python_repl_ast
Action Input: df[(df['Course'] == 'Master of Data Science') & (df['Price'] < 40000)]['University Name']
Observation: 7    University of Queensland
9             RMIT University
Name: University Name, dtype: object
Thought: I now know the final answer
Final Answer: University of Queensland and RMIT University

> Finished chain.


{'input': 'Give me the names of Universities where Master of Data Science course is under the price of 40000',
 'output': 'University of Queensland and RMIT University'}